### Hyperparameter Tuning for Facebook Prophet in Parallel 

In [ ]:
import pandas as pd
import numpy as np
import pprint
import fbprophet
import plotly

from fbprophet.plot import plot_plotly
import plotly.offline as py
py.init_notebook_mode()
from fbprophet.diagnostics import cross_validation
from fbprophet.diagnostics import performance_metrics

from tqdm.autonotebook import tqdm
import functools
import dask
from dask.distributed import Client
from dask.distributed import get_client
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
from itertools import product, cycle
import concurrent.futures
import itertools
# !pip install dask
#!pip install graphviz

In [2]:
url = "https://raw.githubusercontent.com/facebook/prophet/master/examples/example_wp_log_peyton_manning.csv"
df = pd.read_csv(url,  sep=',')
df.head(5)

,ds,y
0,2007-12-10,9.590761
1,2007-12-11,8.519590
2,2007-12-12,8.183677
3,2007-12-13,8.072467
4,2007-12-14,7.893572


In [ ]:
#algorithm ='Newton','LBFGS'

m = fbprophet.Prophet()
m.fit(df, algorithm='LBFGS')
future = m.make_future_dataframe(periods=50)
forecast = m.predict(future)
m.plot(forecast);

In [17]:
def single_param_cv(history_df, metrics, param_dict):
    m = fbprophet.Prophet(**param_dict)
    m.fit(history_df)
    df_cv = cross_validation(m, initial='2600 days', period='100 days', horizon = '200 days')
    df_p = performance_metrics(df_cv, rolling_window=1)
    df_p['params'] = str(param_dict)
    df_p = df_p.loc[:, metrics]
    return df_p

def param_grid_to_df(**param_dict):
    param_iter = itertools.product(*param_dict.values())
    params =[]
    for param in param_iter:
        params.append(param) 
    params_df = pd.DataFrame(params, columns=list(param_dict.keys()))
    return params_df


def hyperparameter_cv(history_df, params_df, single_cv_callable, pool):
    results = []
    for param in params_df.values:
        param_dict = dict(zip(params_df.keys(), param))
        if pool is None:
            predict = single_cv_callable(history_df, param_dict=param_dict)
            results.append(predict)
        elif isinstance(pool, dask.distributed.client.Client):
            remote_df = pool.scatter(history_df)
            future = pool.submit(single_cv_callable, remote_df, param_dict=param_dict)
            results.append(future)
        else:
            raise ValueError(f'Pool needs to be an instantiated dask distributed client object or None')
    if isinstance(pool, dask.distributed.client.Client):
        results = pool.gather(results)
    results_df = pd.concat(results)
    
    return results_df


param_dict = {  
                'changepoint_prior_scale': [0.1, 1, 10],
                'changepoint_range': [0.8, 0.9]
              }

metrics = ['horizon', 'rmse', 'mape', 'params'] 


In [15]:
params_df = param_grid_to_df(**param_dict)
single_cv_callable = functools.partial(single_param_cv, metrics=metrics)

##### Sequential for loop in pandas

In [19]:
%%time

result_df  = hyperparameter_cv(df, params_df, single_cv_callable, pool=None)
best_param = result_df.loc[result_df['rmse'] == min(result_df['rmse']), ['params']]
print(f'The best param combination is {best_param.values[0][0]}')
result_df

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Making 2 forecasts with cutoffs between 2015-03-26 00:00:00 and 2015-07-04 00:00:00


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


INFO:fbprophet:Making 2 forecasts with cutoffs between 2015-03-26 00:00:00 and 2015-07-04 00:00:00


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


INFO:fbprophet:Making 2 forecasts with cutoffs between 2015-03-26 00:00:00 and 2015-07-04 00:00:00


...

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Making 2 forecasts with cutoffs between 2015-03-26 00:00:00 and 2015-07-04 00:00:00


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


INFO:fbprophet:Making 2 forecasts with cutoffs between 2015-03-26 00:00:00 and 2015-07-04 00:00:00



The best param combination is {'changepoint_prior_scale': 0.1, 'changepoint_range': 0.8}
CPU times: user 2min 14s, sys: 6.43 s, total: 2min 21s
Wall time: 2min 25s


,horizon,rmse,mape,params
0,200 days,0.453529,0.034922,"{'changepoint_prior_scale': 0.1, 'changepoint_..."
0,200 days,0.464545,0.036444,"{'changepoint_prior_scale': 0.1, 'changepoint_..."
0,200 days,0.460679,0.035948,"{'changepoint_prior_scale': 1.0, 'changepoint_..."
0,200 days,0.503636,0.040299,"{'changepoint_prior_scale': 1.0, 'changepoint_..."
0,200 days,0.463724,0.036518,"{'changepoint_prior_scale': 10.0, 'changepoint..."
0,200 days,0.505390,0.039331,"{'changepoint_prior_scale': 10.0, 'changepoint..."


#### Using client object and Futures API

In [4]:
pool = Client()
pool

Client Scheduler: tcp://127.0.0.1:50214 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 4.29 GB


In [18]:


%time result_df  = hyperparameter_cv(df, params_df, single_cv_callable, pool=pool)
result_df
best_param = result_df.loc[result_df['rmse'] == min(result_df['rmse']), ['params']]
print(f'The best param combination is {best_param.values[0][0]}')
result_df

The best param combination is {'changepoint_prior_scale': 0.1, 'changepoint_range': 0.8}
CPU times: user 10.8 s, sys: 1.41 s, total: 12.2 s
Wall time: 1min 36s


,horizon,rmse,mape,params
0,200 days,0.453529,0.034922,"{'changepoint_prior_scale': 0.1, 'changepoint_..."
0,200 days,0.464545,0.036444,"{'changepoint_prior_scale': 0.1, 'changepoint_..."
0,200 days,0.460679,0.035948,"{'changepoint_prior_scale': 1.0, 'changepoint_..."
0,200 days,0.503636,0.040299,"{'changepoint_prior_scale': 1.0, 'changepoint_..."
0,200 days,0.463724,0.036518,"{'changepoint_prior_scale': 10.0, 'changepoint..."
0,200 days,0.505390,0.039331,"{'changepoint_prior_scale': 10.0, 'changepoint..."
